# Twitter Trust Correlation

In [0]:
import pandas as pd
import numpy as np
import pickle as pk

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!wget http://muftayebaly.com/dropbox/tweetsCID.csv

--2020-04-09 17:22:26--  http://muftayebaly.com/dropbox/tweetsCID.csv
Resolving muftayebaly.com (muftayebaly.com)... 107.180.20.85
Connecting to muftayebaly.com (muftayebaly.com)|107.180.20.85|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54234968 (52M) [text/csv]
Saving to: ‘tweetsCID.csv’

tweetsCID.csv       100%[===================>]  51.72M  14.6MB/s    in 4.1s    

2020-04-09 17:22:30 (12.7 MB/s) - ‘tweetsCID.csv’ saved [54234968/54234968]



In [6]:
!wget https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_articles.csv

--2020-04-09 17:22:38--  https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_articles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9186850 (8.8M) [text/plain]
Saving to: ‘knight_data_articles.csv’

knight_data_article 100%[===================>]   8.76M  55.5MB/s    in 0.2s    

2020-04-09 17:22:39 (55.5 MB/s) - ‘knight_data_articles.csv’ saved [9186850/9186850]



In [7]:
!wget https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_urls.csv

--2020-04-09 17:23:34--  https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_urls.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9199526 (8.8M) [text/plain]
Saving to: ‘knight_data_urls.csv’

knight_data_urls.cs 100%[===================>]   8.77M  44.5MB/s    in 0.2s    

2020-04-09 17:23:35 (44.5 MB/s) - ‘knight_data_urls.csv’ saved [9199526/9199526]



In [11]:
!wget https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_ratings.csv

--2020-04-09 17:24:10--  https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_ratings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2344804 (2.2M) [text/plain]
Saving to: ‘knight_data_ratings.csv’

knight_data_ratings 100%[===================>]   2.24M  --.-KB/s    in 0.03s   

2020-04-09 17:24:10 (77.2 MB/s) - ‘knight_data_ratings.csv’ saved [2344804/2344804]



In [0]:
raw_data = pd.read_csv("knight_data_articles.csv")
#raw_data.head()

In [0]:
raw_urls = pd.read_csv("knight_data_urls.csv")
#raw_urls.head()

In [13]:
raw_tweets = pd.read_csv("tweetsCID.csv")
raw_tweets

,tweet_id,tweet_text,user_id,tweet_time,no_favs,no_retweets,urls,content_id
0,851485433003421701,#Trump’s border wall would be an ecological an...,613521891,2017-04-10T17:22:01.000Z,0,0,https://www.vox.com/energy-and-environment/201...,2736
1,851514117374398464,#Trump’s border wall would be an ecological an...,222481411,2017-04-10T19:16:00.000Z,0,0,https://www.vox.com/energy-and-environment/201...,2736
2,855009400670617600,The research and experts are clear: Tougher bo...,2347049341,2017-04-20T10:45:01.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4...,2644
3,855009683542728704,Hmmmmmm https://t.co/y3fNx7KuJT,243580387,2017-04-20T10:46:08.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4...,2644
4,855011873183342592,(psst! It was never about drugs. It's always b...,17889654,2017-04-20T10:54:50.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4...,2644
...,...,...,...,...,...,...,...,...
201868,930566176907853824,Big increase in investment hastening arrival o...,454368263,2017-11-14T22:40:40.000Z,0,0,https://www.nytimes.com/2017/11/13/business/se...,3351
201869,930571261775876097,As readers of https://t.co/ajmt36bUQU already ...,61681958,2017-11-14T23:00:52.000Z,0,0,https://tytnetwork.com/Investigates/; https://...,3351
201870,930573435343908864,We’ve been reporting on what FedEx and UPS are...,905632398511046656,2017-11-14T23:09:30.000Z,0,0,https://www.nytimes.com/2017/11/13/business/se...,3351
201871,930582220800233473,Self-Driving Trucks May Be Closer Than They Ap...,831215040,2017-11-14T23:44:25.000Z,0,0,https://www.nytimes.com/2017/11/13/business/se...,3351


In [36]:
raw_tweets.describe()

,tweet_id,user_id,no_favs,no_retweets
count,2.018730e+05,2.018730e+05,201873.0,201873.0
mean,9.207283e+17,1.332248e+17,0.0,0.0
std,7.496690e+15,3.013590e+17,0.0,0.0
min,8.514854e+17,5.700000e+01,0.0,0.0
25%,9.174132e+17,5.645870e+07,0.0,0.0
50%,9.218926e+17,4.205027e+08,0.0,0.0
75%,9.260537e+17,2.853383e+09,0.0,0.0
max,9.305854e+17,9.301665e+17,0.0,0.0


In [14]:
raw_ratings = pd.read_csv("knight_data_ratings.csv")
raw_ratings.head()

,external_user_id,content_id,rating_scale_response,sampdate,blind
0,047641_S02_01,60,4.0,20170901,0
1,4205985366_78419_01,60,5.0,20170901,1
2,4205985366_78419_01,61,4.5,20170901,1
3,4205985366_78419_01,66,3.5,20170901,1
4,4205985366_78419_01,80,4.0,20170901,1


In [0]:
import os
import shutil

## Correlation of \# of Tweets and Trust

In [19]:
blind_ratings = raw_ratings[raw_ratings['blind']==1].groupby(['content_id']).mean()
blind_ratings['content_id1'] = blind_ratings.index
blind_ratings

,rating_scale_response,sampdate,blind,content_id1
content_id,,,,
60,5.000,20170901,1,60
61,4.500,20170901,1,61
66,3.500,20170901,1,66
80,3.750,20170901,1,80
82,3.500,20170901,1,82
...,...,...,...,...
3375,2.500,20170901,1,3375
3378,4.625,20170901,1,3378
3380,3.500,20170901,1,3380


In [22]:
num_tweets = raw_tweets.groupby(['content_id']).count()
num_tweets['content_id1'] = num_tweets.index
num_tweets

,tweet_id,tweet_text,user_id,tweet_time,no_favs,no_retweets,urls,content_id1
content_id,,,,,,,,
101,33,33,33,33,33,33,33,101
1019,14,14,14,14,14,14,14,1019
102,20,20,20,20,20,20,20,102
103,18,18,18,18,18,18,18,103
1060,2,2,2,2,2,2,2,1060
...,...,...,...,...,...,...,...,...
984,14,14,14,14,14,14,14,984
985,273,273,273,273,273,273,273,985
990,8,8,8,8,8,8,8,990


In [34]:
corr_df = pd.DataFrame(columns=['content_id','num_tweets','trust_mean'])

for index, row in num_tweets.iterrows():
  try:
    corr_df = corr_df.append({'content_id':index, 'num_tweets':row['tweet_id'], 'trust_mean':blind_ratings['rating_scale_response'][int(index)]}, ignore_index=True)
  except Exception:
    pass

corr_df

,content_id,num_tweets,trust_mean
0,101,33,4.000000
1,1019,14,1.000000
2,102,20,0.500000
3,103,18,0.500000
4,1060,2,4.000000
...,...,...,...
1250,969,13,2.000000
1251,984,14,4.166667
1252,985,273,2.750000
1253,990,8,2.800000
